In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# import

In [1]:
# export 
import sys 
if '..' not in sys.path:
    sys.path.append('..')
from exp import resnet_ssd

In [2]:
# export
from FLAI.detect_symbol.exp import databunch as databunch_detsym
from FLAI.detect_symbol.exp import init_model as init_model_detsym

In [3]:
# export
import sys
if '..' not in sys.path:
    sys.path.append('..')
from exp import databunch

In [4]:
# export
from fastai.basic_train import Learner,DatasetType

In [5]:
# export
from fastai.callbacks.hooks import Hook,Hooks

In [6]:
# export
from torch import nn

In [7]:
# export
import torch

In [8]:
from IPython.core import debugger as idb

In [9]:
# export
from fastprogress.fastprogress import progress_bar

In [10]:
# export
import re

# functions

# test

In [12]:
device = torch.device('cuda')

In [13]:
#先用detect_symbol的数据集测试
data_detsym = databunch_detsym.get_databunch(data_root = '../../detect_symbol/data/ds_20200429', bs=8, device=device)
x,y = data_detsym.one_batch()

In [16]:
data = databunch.get_databunch(data_root = './ds_20200818', bs=16, device=device)
x,y = data.one_batch()

### resnet18_ssd

In [ ]:
model = resnet_ssd.get_resnet18_1ssd(num_classes = 17)

In [ ]:
# check layer output stats before runtime init.
init_model_detsym.show_layer_stats(model,x)

In [ ]:
# runtime init
init_model_detsym.runtime_init_linear(model, x, init_model_detsym.hook_init);

In [ ]:
# check layer output stats after runtime init
init_model_detsym.show_layer_stats(model,x)

In [ ]:
# 因为runtime_init耗时较长，保存初始化后的模型，方便快速调用
# torch.save(model.state_dict(), './models/resnet18_ssd_init.pth') # 只在要保存初始化模型时运行该行代码

### resnet18_1ssd

In [ ]:
#符号检测的17种符号！！！
#model = resnet_ssd.get_resnet18_1ssd(num_classes = 17)

In [17]:
model = resnet_ssd.get_resnet18_1ssd(num_classes = 1)

#### LSUV

In [18]:
# check layer output stats before runtime init.
init_model_detsym.show_layer_stats(model,x)

(7, torch.Size([64, 388, 388]), -0.006528593134135008, 0.04469042271375656)
(3, torch.Size([64, 194, 194]), -0.3074367642402649, 1.270885944366455)
(3, torch.Size([64, 194, 194]), 0.06614367663860321, 0.9619664549827576)
(3, torch.Size([64, 194, 194]), 0.0077298651449382305, 1.8441429138183594)
(3, torch.Size([64, 194, 194]), 0.06412684172391891, 1.007917046546936)
(3, torch.Size([128, 97, 97]), 0.02510482631623745, 1.5859466791152954)
(3, torch.Size([128, 97, 97]), -0.03685520216822624, 1.0046296119689941)
(1, torch.Size([128, 97, 97]), -0.09126517921686172, 1.644511342048645)
(3, torch.Size([128, 97, 97]), 0.05147907882928848, 1.4021588563919067)
(3, torch.Size([128, 97, 97]), -0.0015884615713730454, 0.9756810069084167)
(3, torch.Size([256, 49, 49]), -0.0397324301302433, 1.2892857789993286)
(3, torch.Size([256, 49, 49]), -0.0283199530094862, 0.9777454137802124)
(1, torch.Size([256, 49, 49]), 0.0014692945405840874, 1.311471700668335)
(3, torch.Size([256, 49, 49]), 0.045558907091617584

In [19]:
# runtime init
init_model_detsym.runtime_init_linear(model, x, init_model_detsym.hook_init);

In [20]:
# check layer output stats after runtime init
init_model_detsym.show_layer_stats(model,x)

(7, torch.Size([64, 388, 388]), -0.06209127977490425, 1.0)
(3, torch.Size([64, 194, 194]), -0.023548142984509468, 1.0)
(3, torch.Size([64, 194, 194]), 0.02986021898686886, 1.0)
(3, torch.Size([64, 194, 194]), 0.05027669668197632, 1.0)
(3, torch.Size([64, 194, 194]), 0.046196553856134415, 1.0)
(3, torch.Size([128, 97, 97]), 0.0028898846358060837, 1.0)
(3, torch.Size([128, 97, 97]), -0.021662622690200806, 1.0)
(1, torch.Size([128, 97, 97]), -0.004570108372718096, 0.9999999403953552)
(3, torch.Size([128, 97, 97]), 0.018538953736424446, 0.9999999403953552)
(3, torch.Size([128, 97, 97]), -0.06961742043495178, 1.0)
(3, torch.Size([256, 49, 49]), -0.009937639348208904, 1.0)
(3, torch.Size([256, 49, 49]), 0.04513965919613838, 0.9999999403953552)
(1, torch.Size([256, 49, 49]), 0.0022508588153868914, 1.0)
(3, torch.Size([256, 49, 49]), 0.02029108814895153, 1.0)
(3, torch.Size([256, 49, 49]), -0.01066808495670557, 1.0)
(1, torch.Size([256, 49, 49]), 0.058665208518505096, 1.0)
(3, torch.Size([2, 4

In [21]:
# 因为runtime_init耗时较长，保存初始化后的模型，方便快速调用
#torch.save(model.state_dict(), './models/resnet18_1ssd_init.pth') # 只在要保存初始化模型时运行该行代码

#### init_to_pretrained

In [22]:
pdict = torch.load('../../detect_symbol/models/pytorch_pretrained/resnet18-5c106cde.pth')

In [23]:
# 将模型中与 resnet18 对应的部分的参数从预训练模型加载
link_names = [(r'^conv1',        'conv1'),
               (r'^bn1',          'bn1'),
               (r'^res_blocks.0', 'layer1'),
               (r'^res_blocks.1', 'layer2'),
               (r'^res_blocks.2', 'layer3'),
               (r'^res_blocks.3', 'layer4')]
init_model_detsym.init_to_pretrained(model, pdict, link_names)

In [24]:
# check layer output stats before runtime init.
init_model_detsym.show_layer_stats(model,x)

(7, torch.Size([64, 388, 388]), 0.002718508243560791, 0.19889268279075623)
(3, torch.Size([64, 194, 194]), -1.0356876850128174, 1.1808650493621826)
(3, torch.Size([64, 194, 194]), -0.07255039364099503, 0.45597755908966064)
(3, torch.Size([64, 194, 194]), -0.7321603894233704, 1.1958472728729248)
(3, torch.Size([64, 194, 194]), -0.04721229150891304, 0.37332046031951904)
(3, torch.Size([128, 97, 97]), -0.3559102416038513, 1.064697504043579)
(3, torch.Size([128, 97, 97]), -0.21354509890079498, 0.35384541749954224)
(1, torch.Size([128, 97, 97]), -0.0739346593618393, 0.4490340054035187)
(3, torch.Size([128, 97, 97]), -0.3113606870174408, 0.5742048025131226)
(3, torch.Size([128, 97, 97]), -0.061193183064460754, 0.22838985919952393)
(3, torch.Size([256, 49, 49]), -0.3419056832790375, 0.5534995794296265)
(3, torch.Size([256, 49, 49]), -0.14468617737293243, 0.34792420268058777)
(1, torch.Size([256, 49, 49]), -0.047240301966667175, 0.1430220603942871)
(3, torch.Size([256, 49, 49]), -0.40989696979

In [25]:
#torch.save(model.state_dict(), './models/pretrained_res18_1ssd.pth') # 只在要保存初始化模型时运行该行代码

## 测试加载之前detect_symbol的训练状态

In [ ]:
fn = '../../detect_symbol/data/ds_20200429/models/run_0.pth'
fn = '../../detect_symbol_bak20200427/data/ds_20200227/models/run_0.pth'
#../detect_symbol/run_log/20200429/run_0.pth
#fn = '../detect_symbol/models/resnet18_1ssd_2_init.pth'
dic = torch.load(fn)

In [ ]:
model2 = resnet_ssd.get_resnet18_1ssd(num_classes = 17)

In [ ]:
for n, p in model2.named_parameters():
    assert p.data.shape == dic['model'][n].data.shape, '%s:%s!=%s' % (n,p.data.shape, dic['model'][n].data.shape)
    print(n, p.data.shape, dic['model'][n].data.shape)
    p.data = dic['model'][n].data.to(torch.device('cpu'))
    #print(dic['model'][n].shape, p.sum().cpu().detach().numpy())
    print(n, p.data.shape, dic['model'][n].data.shape, '====')
    
    

# export

In [45]:
!python ../notebook2script.py --fname 'init_model.ipynb' --outputDir '../exp/'

Converted init_model.ipynb to ../exp/init_model.py
